This is a simple example script to show you how to pull ETH2 staking data by iterating by block

In [14]:
# import packages
import pandas as pd
import requests
from datetime import datetime, timedelta
from typing import Any
from web3 import Web3

In [16]:
# functions to get next dates block from preious date
def get_datetime_from_block(w3, block_number: int) -> datetime:
    """From a block bumber get datetime of creation
    Parameters:
        w3: web3.py connection
        block_number (int): block to get datetime from
    Returns:
        datetime: datetime of cration of block
    """
    timestamp = int(w3.eth.get_block(block_number)['timestamp'])
    date_time = datetime.utcfromtimestamp(timestamp)

    return date_time


def get_next_day_block(w3, last_sync_block: int) -> Any:
    """Caclulates block of the next day from the day of
       last_sync_block
    Parameters:
        w3: web3.py connection
        last_sync_block (int): last block synced
    Returns:
        block (int): new block to sync
        new_day (datetime): datetime of the block
    """
    last_sync_day = get_datetime_from_block(w3, last_sync_block)

    block = last_sync_block
    new_day = last_sync_day

    next_day = last_sync_day + timedelta(days=1)

    next_day = next_day.replace(hour=0, minute=0, second=0)
    while new_day < next_day:
        if (next_day.date() != datetime.utcnow().date()):
            block += 1000
        else:
            block = w3.eth.block_number - 5
        new_day = get_datetime_from_block(w3, block)

        if (new_day.date() > last_sync_day.date()):
            while new_day.hour > 0:
                block -= 40
                new_day = get_datetime_from_block(w3, block)

    return block, new_day


In [15]:
# web3 node url
web3_url = ## your node url here ''
w3 = Web3(Web3.HTTPProvider(web3_url))

In [38]:
# make subgraph graphql call with iteration
df_final = pd.DataFrame(columns=['totalAmountDeposited', 'date'])

url = 'https://api.thegraph.com/subgraphs/name/sc0vu/eth2'

block = 13000000
day = get_datetime_from_block(w3, block)

while day <= datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0):
    query = """
    {
    aggregations(block: { number: __block__}) {
        totalAmountDeposited
        }
    }"""

    query = query.replace('__block__', str(block))
    response = requests.post(url, json={'query': query})

    df = pd.json_normalize(response.json()['data']['aggregations'])
    df['date'] = day
    df_final = pd.concat([df_final, df]).reset_index(drop=True)
    
    block, day = get_next_day_block(w3, block)
    
df_final

,totalAmountDeposited,date
0,6630882000000000,2021-08-10 21:53:39
1,6630978000000000,2021-08-11 00:52:34
2,6657794000000000,2021-08-12 00:59:13
3,6671170000000000,2021-08-13 00:58:03
4,6688194000000000,2021-08-14 00:52:44
...,...,...
63,7934626000000000,2021-10-12 00:57:05
64,7938658000000000,2021-10-13 00:57:41
65,7943074000000000,2021-10-14 00:58:48
66,7949154000000000,2021-10-15 00:54:48


In [39]:
# save to csv
df_final.to_csv('ETH2_staked_daily_sum.csv', index=False)